# Caricamento delle collezioni e collegamento al database
Il Database è composto da 4 collezioni: food, global_dataset, global_inflation, consumer_price_index.
- food: contiene i dati relativi all'inflazione del cibo per i paesi dell'est europa
- global_dataset: contiene i dati relativi all'inflazione per i paesi del mondo divisi per 5 categorie
- global_inflation: contiene i dati relativi all'inflazione per i paesi del mondo divisi per anno
- consumer_price_index: contiene i dati relativi all'indice di prezzo al consumo per i paesi degli Stati Uniti

In [ ]:
import pymongo
import utils 
import pandas as pd
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["ProgettoBD2"]
food = db["food"]
global_dataset = db["global_dataset"]
global_inflation = db["global_inflation"]
us_cpi = db["consumer_price_index"]

## Collection: Food
- Query per estrarre i paesi presenti nella collezione \
I paesi presenti nella collezione sono principalmente paesi dell'est Europa

In [ ]:
countries = food.distinct("country")
countries

## Collection: Global Dataset
Lista dei paesi presenti nella collezione

In [ ]:
countries = global_dataset.distinct("Country")
countries

L'inflazione è suddivisa in categorie.
- Query per la stampa delle categorie presenti nella collezione

In [ ]:
global_dataset.distinct("Series Name")

# Unione Europea 
Si è deciso di analizzare i paesi dell'Unione Europea presenti nella collezione global_dataset

- Estrazione dei paesi dell'Unione Europea presenti nella collezione

In [ ]:
query = {"Country Code": {"$in": ["AUT", "BEL", "BGR", "HRV", "CYP", "CZE", "DNK", "EST", "FIN", "FRA", "DEU", "GRC", "HUN", "IRL", "ITA", "LVA", "LTU", "LUX", "MLT", "NLD", "POL", "PRT", "ROU", "SVK", "SVN", "ESP", "SWE"]}}

europeanCountries = global_dataset.find(query)

europeanCountriesList = global_dataset.distinct("Country", query)
for country in europeanCountriesList:
    print(country)

Query che effettua la media anno per anno del tasso di inflazione nei paesi dell'unione europea di tutte le categorie.
In sostanza vengono eliminate le categorie aggregando i 5 valori per ogni anno in una media. 

In [ ]:
# Esecuzione dell'aggregazione
result = utils.get_avg_infl_eur(global_dataset, europeanCountriesList)
# Stampa dei risultati
for doc in result:
    print(doc)

Analizzata l'inflazione dell'intera Unione Europea negli anni
 

In [ ]:
europeanCountriesList = global_dataset.distinct("Country", query)
result = utils.get_food_inflation_eur_per_year(global_dataset, europeanCountriesList)
output = {}

  # Riempimento del dizionario con i risultati
for doc in result: 
    doc.pop('_id')
    doc.pop('Indicator Type')
    doc.pop('Note')
    output.update(doc)

print(output)

In [ ]:
import matplotlib.pyplot as plt

# Estrazione degli anni e dei valori di inflazione dal dizionario
anni = list(output.keys())
inflazione = list(output.values())

# Creazione del grafico
plt.figure(figsize=(10, 6))
plt.plot(anni, inflazione, marker='o', label='Inflazione annuale')
plt.xlabel('Anno')
plt.ylabel('Inflazione (%)')
plt.title('Inflazione annuale nell\'Unione Europea')
plt.xticks(rotation=90)
plt.legend()
plt.show()

## Integrazione con food: primi passi
Nel Dataset Food non sono presenti molti paesi dell'Unione Europea, per questo motivo si è deciso di integrare i dati di food con quelli di global_dataset. \
Viene eseguita una query per estrarre i dati dell'inflazione dei paesi europei sul cibo da global_dataset sfruttando il campo Series Name per estrarre solo i valori inerenti il campo "Food Consumer Price Inflation". \
 Questa query è stata effettuata utilizzando i Country Code dei paesi europei. 


In [ ]:
#todo: Implementare in app come sotto funzione dell'inflazione sul cibo totale

europeanCountriesList = ["AUT", "BEL", "BGR", "HRV", "CYP", "CZE", "DNK", "EST", "FIN", "FRA", "DEU", "GRC", "HUN", "IRL", "ITA", "LVA", "LTU", "LUX", "MLT", "NLD", "POL", "PRT", "ROU", "SVK", "SVN", "ESP", "SWE"]

result = utils.get_eu_food_infl_countries(global_dataset, europeanCountriesList)

for doc in result:
    doc.pop('_id')
    doc.pop('Indicator Type')
    doc.pop('Note')
    print(doc)


### Query MongoDB per l'integrazione dei dati atta a raggruppare i dati di food per anno
Inizialmente i dati erano raggruppati per mese, è stata effettuata una somma dei singoli valori open, close, high, low per ogni mese e per ogni paese. \n
- Nel dettaglio, prendiamo per esempio il valore low, settando un anno: se un paese aveva n valori low (Uno per ogni mese dell'anno presente) il nuovo valore low viene calcolato sommando gli n valori low. \n


In [ ]:
pipeline = [
    {
        "$addFields": {
            "year": {"$year": "$date"},  # Estrai l'anno dalla data
        }
    },
    {
        "$group": {
            "_id": {"country": "$country", "year": "$year"},  # Raggruppa per country e year
            "openSum": {"$sum": "$Open"},  # Somma i valori di Open
            "highSum": {"$sum": "$High"},  # Somma i valori di High
            "closeSum": {"$sum": "$Close"},  # Somma i valori di Close
            "lowSum": {"$sum": "$Low"},  # Somma i valori di Low
        }
    },
    {
        "$project": {
            "_id": 0,
            "country": "$_id.country",
            "year": "$_id.year",
            "open": "$openSum",
            "high": "$highSum",
            "close": "$closeSum",
            "low": "$lowSum",
        }
    },
]

grouped_data = list(food.aggregate(pipeline))

# Integrazione dei dati completa del data_set food con global Dataset
- Vengono estratti i paesi distinti presenti in global_dataset e in food, viene eseguita la query per ottenere l'inflazione di ogni paese relativa al campo alimentare in global_dataset attraverso il campo Series Name. \n
- Viene in seguito definita una lista dei paesi presenti nel risultato della query e vengono ad essi aggiunti i paesi mancanti presenti nel dataset food con i relativi dati 
- Sono risultati alla fine solo due paesi mancanti: Somalia e South Sudan. I valori dell'inflazione sono stati presi dal valore High del dataset food, che rappresenta il massimo dell'inflazione risultante in quel periodo.

In [ ]:
#todo: Implementare in app

country_codes = global_dataset.distinct("Country Code")
food_country_list = food.distinct("country")

result = list(utils.get_eu_food_infl_countries(global_dataset, country_codes))
grouped_data_dict = {(doc['country'], doc['year']): doc['high'] for doc in grouped_data}
existing_countries = [doc['Country'] for doc in result]

for doc in result:
    doc.pop('_id')
    doc.pop('Indicator Type')
    doc.pop('Note')

for country in food_country_list:
    if country not in existing_countries:
        for year in range(1980, 2025): 
            high_value = grouped_data_dict.get((country, year))
            if high_value is not None:
                result.append({
                    'country': country,
                    'year': year,
                    'infl': high_value
                })
                
for doc in result:
    print(doc)
        

## Collection: Global Inflation
Lista dei paesi presenti nella collezione

In [ ]:
countries = global_inflation.distinct("country_name")
countries

#### Estrazione Paese con maggiore inflazione in base all'anno

In [ ]:
# Query per trovare il paese con la massima inflazione in un anno specifico
anno = 1991
max_inflation = utils.get_max_infl_year(global_inflation, anno)

country_max_infl = global_inflation.find_one({"_id": max_inflation.next()['_id']})
country_name = country_max_infl['country_name']

print(f"Il paese con la maggiore inflazione nel {anno} è: {country_name} con un valore di {country_max_infl[str(anno)]}%")

In [ ]:
del country_max_infl['_id']
del country_max_infl['country_name']
del country_max_infl['indicator_name']

## Plot dell'inflazione annuale per il paese con la maggiore inflazione precedentemente calcolata

In [ ]:
import matplotlib.pyplot as plt

anni = list(country_max_infl.keys())
inflazione = list(country_max_infl.values())
plt.figure(figsize=(10, 6))
plt.plot(anni, inflazione, marker='o', label='Inflazione annuale')
plt.xlabel('Anno')
plt.ylabel('Inflazione (%)')
plt.title(f'Inflazione annuale di: {country_name}')
plt.xticks(rotation=90)
plt.show()

# Inflazione media per paese

In [ ]:
#todo: Implementare in app

country_name = "Italy"
avg_ita_infl = utils.get_avg_infl_years(global_inflation, country_name)
print(f"L'inflazione media in {country_name} dal 1980 al 2024 è {avg_ita_infl.next()['avgInflation']:.2f}%")

# Analisi dell'inflazione negli anni dato il paese 

In [ ]:
country = "Italy"
inflation_values, years = utils.get_inflation_by_country(global_inflation, country)

plt.figure(figsize=(10, 6))
plt.plot(inflation_values, years, marker='o', label='Inflazione annuale')
plt.xlabel('Anno')
plt.ylabel('Inflazione (%)')
plt.title(f'Inflazione annuale di: {country_name}')
plt.xticks(rotation=90)
plt.legend()
plt.show()